# SAN Assignment - Linear discriminant analysis

Author : Your Name \
Email  : you@fel.cvut.cz

#### Introduction
The aim of this assignment is to get familiar with Linear Discriminant Analysis
(LDA). LDA and Principal Component Analysis (PCA) are two techniques
for dimensionality reduction. PCA can be described as an unsupervised algorithm that ignores data labels and aims to find directions which maximalize
the variance in a data. In comparison with PCA, LDA is a supervised algorithm and aims to project a dataset onto a lower dimensional space with good
class separability. In other words, LDA maximalizes the ratio of betweenclass variance and the within-class variance in a given data.

#### Input data 
In this tutorial, we will work with a dataset that classifies wines (samples)
into three classes using of 13 continuous attributes; for more details see
wine info.txt file. The dataset is located at wine.csv.

#### Linear Discriminant Analysis
As we mentioned above, LDA finds directions where classes are well-separated,
i.e. LDA maximizes the ratio of between-class variance and the within-class
variance. Firstly, assume that $C$ is a set of classes and set $D$, which represents
a training dataset, is defined as $D = \{x_1, x_2, . . . , x_N \}$.

The between-classes scatter matrix SB is defined as:
$S_b = \sum_c N_C(\mu_c -\overline{x})(\mu_c - \overline{x})^T$, where $\overline{x}$ is a vector represents the overall mean of the data, µ represents the mean corresponding to each class, and $N_C$ are sizes of the respective classes.

The within-classes scatter matrix $S_W$ is defined as:

$S_W = \sum_c \sum_{x \in D_c}(x - \overline{\mu_c})(x - \overline{\mu_c})^T$

Next, we will solve the generalized eigenvalue problem for the matrix $S_W^{-1}S_B$ to obtain the linear discriminants, i.e.

$(S_W^{-1}S_B)w = \lambda w$

where $w$ represents an eigenvector and $\lambda$ represents an eigenvalue. Finally,
choose k eigenvectors with the largest eigenvalue and transform the samples
onto the new subspace.


##### imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

#### Step by step

##### Load the dataset

##### Compute the within-scatter matrix

In [ ]:
def compute_within_scatter(data):
    """ Function to compute within scatter matrix S_w.

    :param data: list of numpy arrays with data from individual classes
    """
    # todo: replace with your code
    within_matrix = np.eye(data[0].shape[1])

    return within_matrix

##### Compute the between-scatter matrix

In [ ]:
def compute_between_scatter(data):
    """ Function to compute between scatter matrix S_b.

    :param data: list of numpy arrays with data from individual classes
    """
    # todo: replace with your code
    between_matrix = np.eye(data[0].shape[1])

    return between_matrix

##### Solve the EigenProblem and return eigen-vector

In [ ]:
def solve_eigen_problem(within_matrix, between_matrix):
    """ Compute eigenvectors as defined by the method.

    Carefully with the datatypes here - numpy will give you complex numbers.
    This does not affect the functionallity, but it does give you a ton of warnings.

    :param within_matrix: numpy array (n_variables, n_variables)
    :param between_matrix: numpy array, same shape
    :return: real eigenvectors, ordered from the highest eigenvalue, numpy array, same shape
    """

    # todo: replace with your code
    eigenvectors = None

    return eigenvectors

##### Visualize the results
Project your data into lower-dimensional subspace, visualize this projection, and compare with PCA (see Fig. 1). Use the following code while filling in the lines marked as `TODO`.

In [ ]:
def split_by_class(data, labels):
    ret = []
    for lbl in np.unique(labels):
        ret.append(data[labels == lbl])
    return ret


def compute_centroids(data, labels):
    ret = []
    for lbl in np.unique(labels):
        ret.append(data[labels == lbl].mean(axis=0))
    return np.array(ret)


def classify(data, eigen_vectors, centroids, prior):
    n_classes = len(centroids)

    y = data @ eigen_vectors[:, :n_classes]
    classif = np.empty((data.shape[0], n_classes))
    for c in range(n_classes):
        classif[:, c] = ((y - centroids[c]) ** 2 / 2).sum(axis=1) - np.log(prior[c])
    return classif.argmin(axis=1) + 1  # indexes start 0, labels start as 1


In [ ]:
def LDA(data, labels):
    data_per_class = split_by_class(data, labels)

    # 1) scatter matrices
    # within-class scatter matrix Sw
    S_w = compute_within_scatter(data_per_class)

    # between-class scatter matrix Sb
    S_b = compute_between_scatter(data_per_class)

    # 2)  eigen problem
    # solve eigen problem
    eigenvectors = solve_eigen_problem(S_w, S_b)

    #transform the samples onto the new subspace
    y = data @ eigenvectors[:, :2]

    ## visual comparison with PCA
    scaled_data = StandardScaler().fit_transform(data)
    y_pca = PCA(2).fit_transform(data)
    y_pca_scaled = PCA(2).fit_transform(scaled_data)

    fig, axs = plt.subplots(ncols=3)
    axs[0].scatter(y[:, 0], y[:, 1], c=labels)
    axs[0].set_xlabel("LDA_1")
    axs[0].set_ylabel("LDA_2")
    axs[0].set_title("LDA")

    axs[1].scatter(y_pca[:, 0], y_pca[:, 1], c=labels)
    axs[1].set_xlabel("PCA_1")
    axs[1].set_ylabel("PCA_2")
    axs[1].set_title("PCA not scaled")

    axs[2].scatter(y_pca_scaled[:, 0], y_pca_scaled[:, 1], c=labels)
    axs[2].set_xlabel("PCA_1")
    axs[2].set_ylabel("PCA_2")
    axs[2].set_title("PCA scaled")

    fig.tight_layout()
    fig.show()

    return eigenvectors


In [ ]:
def crossvalidation_lda(data, labels, n_folds=10):
    seed = 0
    n_classes = len(np.unique(labels))
    accuracy = []
    skf = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
    for train_idxs, test_idxs in skf.split(data, labels):
        eigen_lda = LDA(data[train_idxs], labels[train_idxs])
        projected_data = data[train_idxs] @ eigen_lda[:, :n_classes]
        centroids = compute_centroids(projected_data, labels[train_idxs])
        predicted_labels = classify(data[test_idxs], eigen_lda, centroids,
                                    np.full(n_classes, 1. / n_classes))
        accuracy.append(np.mean(predicted_labels == labels[test_idxs]))
    return np.mean(accuracy)

In [ ]:
############################# MAIN ##########################################

df = pd.read_csv("wine.csv", header=None)
labels = df.iloc[:, 0].values
data = df.iloc[:, 1:].values

n_classes = len(np.unique(labels))

#compute LDA and return corresponding eigenvectors
eigen_lda = LDA(data, labels)

projected_data = data @ eigen_lda[:, :n_classes]
centroids = compute_centroids(projected_data, labels)

predicted_labels = classify(data, eigen_lda, centroids, np.full(n_classes, 1. / n_classes))

#ACC
print("accuracy:", np.mean(predicted_labels == labels))

#CrossValidation
acc_lda = crossvalidation_lda(data, labels, n_folds=10)

##### Discuss given results and copare the methods.
